In [1]:
import json
import pandas as pd
import re
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')

In [2]:
with open('result.json', encoding='utf8') as f:
    data = json.load(f)

In [3]:
def get_msg_df(df):
    msg_df = df[df['type']=='message'].filter(items = ['id','from','text','from_id']).reset_index(drop=True)
    tmp_msg_df = []
    for index, row in msg_df.iterrows():
        if type(row['text']) == list:
            s = ''
            for item in row['text']:
                if type(item) == str:
                    s += item
                elif item['type'] != 'link':
                    s += item['text']
            row['text'] = s
        s = ''
        for c in row['text']:
            c = c.lower()
            if c.isalpha() or c.isdigit() or c == "'":
                s += c
            elif c == '?':
                s += ' '+c+' '
            else:
                s += ' '
        row['text'] = ' '.join(s.split())
        tmp_msg_df.append(row)

    tmp_msg_df = pd.DataFrame(tmp_msg_df).reset_index(drop=True)
    msg_df['clean_text'] = tmp_msg_df['text']
    msg_df = msg_df[msg_df['clean_text'] != ''].reset_index(drop=True)
    return msg_df

In [4]:
def get2_msg_df(df):

    def preprocessing_msg_df(msg_df):
        new_msg_df = []
        for index, row in msg_df.iterrows():
            if type(row['text']) == list:
                s = ''
                for item in row['text']:
                    if type(item) == str:
                        s += item
                    else:
                        s += item['text']
                row['text'] = s
            if row['text'] != '':
                new_msg_df.append(row)
        new_msg_df = pd.DataFrame(new_msg_df).reset_index(drop=True)
        return new_msg_df

    def cleaning_msg_df(msg_df):
        msg_df['clean_text'] = msg_df['text'].str.lower()
        msg_df['clean_text'] = msg_df['clean_text'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http+?", "", elem))  
        return msg_df
    
    msg_df = df[df['type']=='message'].filter(items = ['id','from','text','from_id']).reset_index(drop=True)
    msg_df = preprocessing_msg_df(msg_df)
    msg_df = cleaning_msg_df(msg_df)
    msg_df = msg_df[msg_df['clean_text'] != ''].reset_index(drop=True)
    return msg_df

In [5]:
df = pd.DataFrame(data['messages'])
msg_df = get2_msg_df(df)

In [6]:
msg_df

,id,from,text,from_id,clean_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys
2,47,Apple J,GM,user1776264703,gm
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys
...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...


In [7]:
stop = stopwords.words('english')
msg_df['stopped_text'] = msg_df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
msg_df

,id,from,text,from_id,clean_text,stopped_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...,10 players battle royale imagine open 100 players
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys,heres real questionwen beta keys
2,47,Apple J,GM,user1776264703,gm,gm
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...,hey guys kurtney glock ventureswhom contact in...
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys,hey guys
...,...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...,anyone wish use difference borrower except the...
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information,thks much information
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...,user log display message tried log reason anyone
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...,guys bug occurring android version think first...


In [9]:
msg_df['token_text'] = msg_df['stopped_text'].apply(lambda x: word_tokenize(x))

In [10]:
msg_df

,id,from,text,from_id,clean_text,stopped_text,token_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...,10 players battle royale imagine open 100 players,"[10, players, battle, royale, imagine, open, 1..."
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys,heres real questionwen beta keys,"[heres, real, questionwen, beta, keys]"
2,47,Apple J,GM,user1776264703,gm,gm,[gm]
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...,hey guys kurtney glock ventureswhom contact in...,"[hey, guys, kurtney, glock, ventureswhom, cont..."
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys,hey guys,"[hey, guys]"
...,...,...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...,anyone wish use difference borrower except the...,"[anyone, wish, use, difference, borrower, exce..."
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information,thks much information,"[thks, much, information]"
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...,user log display message tried log reason anyone,"[user, log, display, message, tried, log, reas..."
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...,guys bug occurring android version think first...,"[guys, bug, occurring, android, version, think..."


In [11]:
def word_stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text
msg_df['stem_text'] = msg_df['token_text'].apply(lambda x: word_stemmer(x))

In [12]:
msg_df

,id,from,text,from_id,clean_text,stopped_text,token_text,stem_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...,10 players battle royale imagine open 100 players,"[10, players, battle, royale, imagine, open, 1...","[10, player, battl, royal, imagin, open, 100, ..."
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys,heres real questionwen beta keys,"[heres, real, questionwen, beta, keys]","[here, real, questionwen, beta, key]"
2,47,Apple J,GM,user1776264703,gm,gm,[gm],[gm]
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...,hey guys kurtney glock ventureswhom contact in...,"[hey, guys, kurtney, glock, ventureswhom, cont...","[hey, guy, kurtney, glock, ventureswhom, conta..."
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys,hey guys,"[hey, guys]","[hey, guy]"
...,...,...,...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...,anyone wish use difference borrower except the...,"[anyone, wish, use, difference, borrower, exce...","[anyon, wish, use, differ, borrow, except, the..."
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information,thks much information,"[thks, much, information]","[thk, much, inform]"
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...,user log display message tried log reason anyone,"[user, log, display, message, tried, log, reas...","[user, log, display, messag, tri, log, reason,..."
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...,guys bug occurring android version think first...,"[guys, bug, occurring, android, version, think...","[guy, bug, occur, android, version, think, fir..."


In [13]:
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
msg_df['lemma_text'] = msg_df['token_text'].apply(lambda x: word_lemmatizer(x))

In [14]:
msg_df

,id,from,text,from_id,clean_text,stopped_text,token_text,stem_text,lemma_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...,10 players battle royale imagine open 100 players,"[10, players, battle, royale, imagine, open, 1...","[10, player, battl, royal, imagin, open, 100, ...","[10, player, battle, royale, imagine, open, 10..."
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys,heres real questionwen beta keys,"[heres, real, questionwen, beta, keys]","[here, real, questionwen, beta, key]","[here, real, questionwen, beta, key]"
2,47,Apple J,GM,user1776264703,gm,gm,[gm],[gm],[gm]
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...,hey guys kurtney glock ventureswhom contact in...,"[hey, guys, kurtney, glock, ventureswhom, cont...","[hey, guy, kurtney, glock, ventureswhom, conta...","[hey, guy, kurtney, glock, ventureswhom, conta..."
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys,hey guys,"[hey, guys]","[hey, guy]","[hey, guy]"
...,...,...,...,...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...,anyone wish use difference borrower except the...,"[anyone, wish, use, difference, borrower, exce...","[anyon, wish, use, differ, borrow, except, the...","[anyone, wish, use, difference, borrower, exce..."
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information,thks much information,"[thks, much, information]","[thk, much, inform]","[thks, much, information]"
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...,user log display message tried log reason anyone,"[user, log, display, message, tried, log, reas...","[user, log, display, messag, tri, log, reason,...","[user, log, display, message, tried, log, reas..."
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...,guys bug occurring android version think first...,"[guys, bug, occurring, android, version, think...","[guy, bug, occur, android, version, think, fir...","[guy, bug, occurring, android, version, think,..."


In [15]:
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text
msg_df['pos_text'] = msg_df['token_text'].apply(lambda x: word_pos_tagger(x))

In [16]:
msg_df

,id,from,text,from_id,clean_text,stopped_text,token_text,stem_text,lemma_text,pos_text
0,28,Mr Pig,"This is 10 players Battle Royale, imagine when...",user194359594,this is 10 players battle royale imagine when ...,10 players battle royale imagine open 100 players,"[10, players, battle, royale, imagine, open, 1...","[10, player, battl, royal, imagin, open, 100, ...","[10, player, battle, royale, imagine, open, 10...","[(10, CD), (players, NNS), (battle, NN), (roya..."
1,29,Legendary Harry Pottre,Here's the real question...\n\nWen beta keys,user1630446861,heres the real questionwen beta keys,heres real questionwen beta keys,"[heres, real, questionwen, beta, keys]","[here, real, questionwen, beta, key]","[here, real, questionwen, beta, key]","[(heres, NNS), (real, JJ), (questionwen, JJ), ..."
2,47,Apple J,GM,user1776264703,gm,gm,[gm],[gm],[gm],"[(gm, NN)]"
3,51,K S,"Hey guys, this is Kurtney from Glock Ventures....",user656383476,hey guys this is kurtney from glock ventureswh...,hey guys kurtney glock ventureswhom contact in...,"[hey, guys, kurtney, glock, ventureswhom, cont...","[hey, guy, kurtney, glock, ventureswhom, conta...","[hey, guy, kurtney, glock, ventureswhom, conta...","[(hey, NN), (guys, NNS), (kurtney, VBP), (gloc..."
4,53,Aaron Brentwood,"Hey Guys,",user1866130249,hey guys,hey guys,"[hey, guys]","[hey, guy]","[hey, guy]","[(hey, NN), (guys, NNS)]"
...,...,...,...,...,...,...,...,...,...,...
49048,117899,Nalma ~Will Never PM For Fund$~,"Anyone you wish to use, no difference between ...",user1222051294,anyone you wish to use no difference between t...,anyone wish use difference borrower except the...,"[anyone, wish, use, difference, borrower, exce...","[anyon, wish, use, differ, borrow, except, the...","[anyone, wish, use, difference, borrower, exce...","[(anyone, NN), (wish, JJ), (use, NN), (differe..."
49049,117900,Oryan94,thks very much for information,user1977369058,thks very much for information,thks much information,"[thks, much, information]","[thk, much, inform]","[thks, much, information]","[(thks, RB), (much, JJ), (information, NN)]"
49050,117902,Jany,"""User can not log in"" this is the display mess...",user616497746,user can not log in this is the display messag...,user log display message tried log reason anyone,"[user, log, display, message, tried, log, reas...","[user, log, display, messag, tri, log, reason,...","[user, log, display, message, tried, log, reas...","[(user, JJ), (log, NN), (display, NN), (messag..."
49051,117906,Gabriele,Guys I just had a bug occurring on the android...,user59544811,guys i just had a bug occurring on the android...,guys bug occurring android version think first...,"[guys, bug, occurring, android, version, think...","[guy, bug, occur, android, version, think, fir...","[guy, bug, occurring, android, version, think,...","[(guys, NNS), (bug, VBP), (occurring, VBG), (a..."


In [17]:
acc_df = msg_df.groupby('from_id', as_index=False).aggregate(list)

In [18]:
acc_df

,from_id,id,from,text,clean_text,stopped_text,token_text,stem_text,lemma_text,pos_text
0,channel1163806960,"[93360, 93383, 93390, 93397, 93399, 93415, 934...","[Monno, Monno, Monno, Monno, Monno, Monno, Mon...",[how does recruitment work?\nYour ape can make...,[how does recruitment workyour ape can make ba...,"[recruitment workyour ape make baby, read unde...","[[recruitment, workyour, ape, make, baby], [re...","[[recruit, workyour, ape, make, babi], [read, ...","[[recruitment, workyour, ape, make, baby], [re...","[[(recruitment, NN), (workyour, NN), (ape, NN)..."
1,channel1209560118,[65994],[Dota 2],[I have 7 WL Cantina 😅],[i have 7 wl cantina ],[7 wl cantina],"[[7, wl, cantina]]","[[7, wl, cantina]]","[[7, wl, cantina]]","[[(7, CD), (wl, JJ), (cantina, NN)]]"
2,channel1337493381,"[100173, 100234, 100240, 100325, 100327, 10045...","[Manez, Manez, Manez, Manez, Manez, Manez, Man...","[Where can we see our crossbow kill count?, ki...","[where can we see our crossbow kill count, kil...","[see crossbow kill count, killswins longer cou...","[[see, crossbow, kill, count], [killswins, lon...","[[see, crossbow, kill, count], [killswin, long...","[[see, crossbow, kill, count], [killswins, lon...","[[(see, VB), (crossbow, NN), (kill, VB), (coun..."
3,channel1364388189,"[88268, 88384, 89311, 89317, 89910, 89919, 899...","[Ricardi, Ricardi, Ricardi, Ricardi, Ricardi, ...",[Please in the SFP01 is it nft with the 1.2% o...,[please in the sfp01 is it nft with the 12 obl...,"[please sfp01 nft 12 obligatorily, whitepaper,...","[[please, sfp01, nft, 12, obligatorily], [whit...","[[pleas, sfp01, nft, 12, obligatorili], [white...","[[please, sfp01, nft, 12, obligatorily], [whit...","[[(please, VB), (sfp01, JJ), (nft, JJ), (12, C..."
4,channel1383910050,[88147],[Ama Pronos.],[Can we buy on secondary market already? The box],[can we buy on secondary market already the box],[buy secondary market already box],"[[buy, secondary, market, already, box]]","[[buy, secondari, market, alreadi, box]]","[[buy, secondary, market, already, box]]","[[(buy, VB), (secondary, JJ), (market, NN), (a..."
...,...,...,...,...,...,...,...,...,...,...
3961,user994913545,[6659],[wukapdoc],[Yo],[yo],[yo],[[yo]],[[yo]],[[yo]],"[[(yo, NN)]]"
3962,user99710390,"[111993, 113049]","[Vincenzo, Vincenzo]","[look at here, Alt+enter]","[look at here, altenter]","[look, altenter]","[[look], [altenter]]","[[look], [altent]]","[[look], [altenter]]","[[(look, NN)], [(altenter, NN)]]"
3963,user999505965,"[91910, 92126, 92134, 94549, 96808]","[Jean-Noël ¤ jnXmas, Jean-Noël ¤ jnXmas, Jean-...",[Did anyone got the answer on the hodler perk ...,[did anyone got the answer on the hodler perk ...,"[anyone got answer hodler perk, fyi saw messag...","[[anyone, got, answer, hodler, perk], [fyi, sa...","[[anyon, got, answer, hodler, perk], [fyi, saw...","[[anyone, got, answer, hodler, perk], [fyi, sa...","[[(anyone, NN), (got, VBD), (answer, JJR), (ho..."
3964,user999686349,"[26735, 26736, 26741, 26742, 26743, 26744, 267...","[Posto ⚡️, Posto ⚡️, Posto ⚡️, Posto ⚡️, Posto...",[80% of the people on all Elrond Telegram Grou...,[80 of the people on all elrond telegram group...,"[80 people elrond telegram groups failed kyc, ...","[[80, people, elrond, telegram, groups, failed...","[[80, peopl, elrond, telegram, group, fail, ky...","[[80, people, elrond, telegram, group, failed,...","[[(80, CD), (people, NNS), (elrond, VBP), (tel..."


In [19]:
msg_df.to_csv('msg_df.csv', index=False)
acc_df.to_csv('acc_df.csv', index=False)